In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Allergies"
cohort = "GSE205151"

# Input paths
in_trait_dir = "../../input/GEO/Allergies"
in_cohort_dir = "../../input/GEO/Allergies/GSE205151"

# Output paths
out_data_file = "../../output/preprocess/Allergies/GSE205151.csv"
out_gene_data_file = "../../output/preprocess/Allergies/gene_data/GSE205151.csv"
out_clinical_data_file = "../../output/preprocess/Allergies/clinical_data/GSE205151.csv"
json_path = "../../output/preprocess/Allergies/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Functional Immunophenotyping of Children with Critical Status Asthmaticus Identifies Differential Gene Expression Responses in Neutrophils Exposed to a Poly(I:C) Stimulus"
!Series_summary	"We determined whether we could identify clusters of children with critical asthma by functional immunophenotyping using an intracellular viral analog stimulus."
!Series_summary	"We performed a single-center, prospective, observational cohort study of 43 children ages 6 – 17 years admitted to a pediatric intensive care unit for an asthma attack between July 2019 to February 2021."
!Series_overall_design	"Neutrophils were isolated from children, stimulated overnight with LyoVec poly(I:C), and mRNA was analyzed using a targeted Nanostring immunology array. Network analysis of the differentially expressed transcripts for the paired LyoVec poly(I:C) samples was performed."
Sample Characteristics Dictionary:
{0: ['polyic_stimulation: Unstimulated', 'polyic_stimulation

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data (mRNA analyzed using Nanostring immunology array)
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the Sample Characteristics Dictionary, we have:
# - Key 0: 'polyic_stimulation' (Unstimulated, Stimulated, No)
# - Key 1: 'cluster' (1, 2, nan)

# For the allergy trait (asthma in this case), we can use the 'cluster' field
# The study mentions clusters of children with critical asthma
trait_row = 1

# Age and gender are not available in the sample characteristics dictionary
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait (cluster) to binary value (0 or 1)"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon and strip whitespace
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert cluster values to binary (0 for cluster 1, 1 for cluster 2)
    try:
        cluster = int(value)
        if cluster == 1:
            return 0
        elif cluster == 2:
            return 1
        else:
            return None
    except (ValueError, TypeError):
        return None

def convert_age(value):
    """Convert age to continuous value (not used in this dataset)"""
    return None

def convert_gender(value):
    """Convert gender to binary value (not used in this dataset)"""
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    try:
        # Look for the sample characteristics data which should be available from previous steps
        # Each cohort typically has a characteristics.csv file from GEO processing
        clinical_data_file = os.path.join(in_cohort_dir, "characteristics.csv")
        clinical_data = pd.read_csv(clinical_data_file, index_col=0)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        clinical_preview = preview_df(selected_clinical_df)
        print("Clinical Data Preview:")
        print(clinical_preview)
        
        # Save the clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except FileNotFoundError:
        print(f"Clinical data file not found. Unable to extract clinical features.")


Clinical data file not found. Unable to extract clinical features.


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['ABCB1', 'ABCF1', 'ABL1', 'ADA', 'AHR', 'AICDA', 'AIRE', 'ALAS1', 'APP',
       'ARG1', 'ARG2', 'ARHGDIB', 'ATG10', 'ATG12', 'ATG16L1', 'ATG5', 'ATG7',
       'ATM', 'B2M', 'B3GAT1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be standard human gene symbols (like ABCB1, ATG5, B2M)
# They follow the standard HGNC gene nomenclature and are recognizable as common human genes
# No mapping is needed as they are already in the preferred format

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("Loading the original clinical data...")
# Get the matrix file again to ensure we have the proper data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

print("Extracting clinical features...")
# Use the clinical_data obtained directly from the matrix file
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data using the normalized gene data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Final validation
note = "Dataset contains gene expression data from patients with Essential Thrombocythemia (ET), Polycythemia Vera (PV), and Primary Myelofibrosis (PMF)."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")

Normalizing gene symbols...
Gene data shape after normalization: (576, 144)
Normalized gene data saved to ../../output/preprocess/Allergies/gene_data/GSE205151.csv
Loading the original clinical data...
Extracting clinical features...
Clinical data preview:
{'GSM6205808': [0.0], 'GSM6205809': [0.0], 'GSM6205810': [1.0], 'GSM6205811': [1.0], 'GSM6205812': [0.0], 'GSM6205813': [0.0], 'GSM6205814': [1.0], 'GSM6205815': [1.0], 'GSM6205816': [1.0], 'GSM6205817': [1.0], 'GSM6205818': [1.0], 'GSM6205819': [1.0], 'GSM6205820': [0.0], 'GSM6205821': [0.0], 'GSM6205822': [1.0], 'GSM6205823': [1.0], 'GSM6205824': [1.0], 'GSM6205825': [1.0], 'GSM6205826': [1.0], 'GSM6205827': [1.0], 'GSM6205828': [0.0], 'GSM6205829': [0.0], 'GSM6205830': [1.0], 'GSM6205831': [1.0], 'GSM6205832': [1.0], 'GSM6205833': [1.0], 'GSM6205834': [0.0], 'GSM6205835': [0.0], 'GSM6205836': [0.0], 'GSM6205837': [0.0], 'GSM6205838': [0.0], 'GSM6205839': [0.0], 'GSM6205840': [1.0], 'GSM6205841': [1.0], 'GSM6205842': [0.0], 'GSM620

For the feature 'Allergies', the least common label is '0.0' with 69 occurrences. This represents 48.25% of the dataset.
The distribution of the feature 'Allergies' in this dataset is fine.



Dataset usability: True
Linked data saved to ../../output/preprocess/Allergies/GSE205151.csv
